In [1]:
from os.path import join as pathjoin
from data_processing import *
from interpretation import *
from models import *
from training import *

Let's download all the data.

In [2]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
MODEL_ID = 'allennlp_bert_base_uncased'
CHECKPOINTS_DIR = pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints')
BEST_MODEL = pathjoin(CHECKPOINTS_DIR, 'best.th')

In [3]:
transformer_model = 'bert-base-uncased'
MAX_TOKENS = 512

In [4]:
generated = pd.read_csv(pathjoin(DATA_DIR, 'min_gpt_bpe/en'))
real = pd.read_csv(pathjoin(DATA_DIR, 'en_train'))
generated['domain'] = 'gen'
real['domain'] = 'real'
all_train_data = generated.append(real).sample(frac=1, random_state=179)
all_train_data.to_csv(pathjoin(DATA_DIR, 'en_adversarial_train.csv'))
all_train_data.head()

,Unnamed: 0,target,text,domain
793,1547,A22,cartier replica A1000 Bucherer auto initiative...,real
487,793,A14,Nature of problem : The analytical calculation...,real
540,540,A11,locate antaitalt at risk - free evaluation bet...,gen
734,734,A12,791 - 16 - 16 - 14 . Tel : + 44 ( 0 ) 1434 685...,gen
176,1334,A8,Both the Executive Branch and Congress play le...,real


In [5]:
train_dataset_reader = build_adversarial_dataset_reader(transformer_model, lower=True)
val_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)

train_data, dev_data = read_data(
    pathjoin(DATA_DIR, "en_adversarial_train.csv"), 
    pathjoin(DATA_DIR, "en_test"),
    train_dataset_reader, 
    val_dataset_reader
)

vocab = build_vocab(train_data + dev_data)

train_data.index_with(vocab)
dev_data.index_with(vocab)

Reading data
<class 'data_processing.AdversarialDatasetReader'> /home/mlepekhin/data/en_adversarial_train.csv



Building the vocabulary


In [8]:
model = build_adversarial_transformer_model(vocab, transformer_model)

Building the model


In [9]:
if torch.cuda.is_available():
    cuda_device = 2
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
print(cuda_device)

2


In [10]:
!rm -rf {CHECKPOINTS_DIR}
!mkdir -p {CHECKPOINTS_DIR}

In [11]:
train_loader, dev_loader = build_data_loaders(train_data, dev_data)

# You obviously won't want to create a temporary file for your training
# results, but for execution in binder for this course, we need to do this.

trainer = build_classifier_trainer(
    model,
    pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints'),
    train_loader,
    dev_loader,
    10,
    cuda_device=cuda_device
)
print("Starting training")
trainer.train()
print("Finished training")

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


Starting training


TypeError: forward() got an unexpected keyword argument 'domain'

In [9]:
!rm "{CHECKPOINTS_DIR}"/*.json "{CHECKPOINTS_DIR}"/model* "{CHECKPOINTS_DIR}"/training*
!ls -lh "{CHECKPOINTS_DIR}"

total 421M
-rw-rw-r-- 1 mlepekhin mlepekhin 420M авг 19 13:28 best.th
drwxrwxr-x 4 mlepekhin mlepekhin 4,0K авг 19 13:23 log
